# Template 

- Author: Israel Oliveira [\[e-mail\]](mailto:'Israel%20Oliveira%20'<prof.israel@gmail.com>)

In [1]:
!pip3 install -U random-forest-mc

In [2]:
%load_ext watermark

In [3]:
import warnings
import pandas as pd
import numpy as np
from random_forest_mc.model import RandomForestMC
from random_forest_mc.utils import load_file_json, dump_file_json, LoadDicts
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, recall_score
from collections import Counter
import hashlib
from joblib import dump

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

In [4]:
from tqdm import tqdm

# from glob import glob

# import matplotlib.pyplot as plt
# %matplotlib inline
# from matplotlib import rcParams
# from cycler import cycler

# rcParams['figure.figsize'] = 12, 8 # 18, 5
# rcParams['axes.spines.top'] = False
# rcParams['axes.spines.right'] = False
# rcParams['axes.grid'] = True
# rcParams['axes.prop_cycle'] = cycler(color=['#365977'])
# rcParams['lines.linewidth'] = 2.5

# import seaborn as sns
# sns.set_theme()

# pd.set_option("max_columns", None)
# pd.set_option("max_rows", None)
# pd.set_option('display.max_colwidth', None)

from IPython.display import Markdown, display
def md(arg):
    display(Markdown(arg))

# from pandas_profiling import ProfileReport
# #report = ProfileReport(#DataFrame here#, minimal=True)
# #report.to

# import pyarrow.parquet as pq
# #df = pq.ParquetDataset(path_to_folder_with_parquets, filesystem=None).read_pandas().to_pandas()

# import json
# def open_file_json(path,mode='r',var=None):
#     if mode == 'w':
#         with open(path,'w') as f:
#             json.dump(var, f)
#     if mode == 'r':
#         with open(path,'r') as f:
#             return json.load(f)

# import functools
# import operator
# def flat(a):
#     return functools.reduce(operator.iconcat, a, [])

# import json
# from glob import glob
# from typing import NewType


# DictsPathType = NewType("DictsPath", str)


# def open_file_json(path):
#     with open(path, "r") as f:
#         return json.load(f)

# class LoadDicts:
#     def __init__(self, dict_path: DictsPathType = "./data"):
#         Dicts_glob = glob(f"{dict_path}/*.json")
#         self.List = []
#         self.Dict = {}
#         for path_json in Dicts_glob:
#             name = path_json.split("/")[-1].replace(".json", "")
#             self.List.append(name)
#             self.Dict[name] = open_file_json(path_json)
#             setattr(self, name, self.Dict[name])


In [5]:
# Run this cell before close.
%watermark -d --iversion -b -r -g -m -v
!cat /proc/cpuinfo |grep 'model name'|head -n 1 |sed -e 's/model\ name/CPU/'
!free -h |cut -d'i' -f1  |grep -v total

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.27.0

Compiler    : GCC 10.2.1 20210110
OS          : Linux
Release     : 5.11.0-7620-generic
Machine     : x86_64
Processor   : 
CPU cores   : 8
Architecture: 64bit

Git hash: 7ce45ccf6a6f93b9c972a78ec4f0c573e070c45e

Git repo: https://github.com/ysraell/random-forest-mc.git

Git branch: main

pandas: 1.3.2
numpy : 1.20.3

CPU	: Intel(R) Xeon(R) CPU E3-1241 v3 @ 3.50GHz
Mem:            31G
Swap:          4.0G


In [6]:
dicts = LoadDicts("../tests/")
dataset_dict = dicts.datasets_metadata
dataset_dict.keys()

dict_keys(['titanic', 'iris', 'creditcard', 'creditcard_trans_int', 'creditcard_trans_float'])

In [7]:
def dfany2dfnumeric(dfA, dfB):
    for col in dfA.columns:
        mapper = {val: i for i,val in enumerate(dfA[col].unique().tolist()+dfB[col].unique().tolist())}
        dfA[col] = dfA[col].apply(lambda x: mapper[x])
        dfB[col] = dfB[col].apply(lambda x: mapper[x])
    return dfA, dfB

# Tinanic

In [8]:
ds_name = "titanic"
params = dataset_dict[ds_name]
dataset = (
    pd.read_csv(params["csv_path"].replace('~','/work'))[params["ds_cols"] + [params["target_col"]]]
    .dropna()
    .reset_index(drop=True)
)
dataset["Age"] = dataset["Age"].astype(np.uint8)
dataset["SibSp"] = dataset["SibSp"].astype(np.uint8)
dataset["Pclass"] = dataset["Pclass"].astype(str)
dataset["Fare"] = dataset["Fare"].astype(np.uint32)
target_col = params['target_col']
dataset = dataset[params['ds_cols']+[target_col]].dropna().reset_index(drop=True)
dataset[target_col] = dataset[target_col].astype(str)

test_size = 0.5
max_test_size = 10000
max_train_size = 10000

split_seeds = [43, 47, 53, 59]
#def fullCycleAnalysis(dataset,params,test_size_frac, max_test_size, max_train_size, split_seeds, )
target_col = params['target_col']
ds_cols = params["ds_cols"]

Results = []
# Each seed is a experiment.
for seed in split_seeds:
    
    df_train_list = []
    df_test_list = []
    df_classes = [dataset.query(f'{target_col} == "{target_val}"').reset_index(drop=True).copy() for target_val in dataset[target_col].unique()]
    while df_classes:
        df = df_classes.pop(0)
        df_train, df_test = train_test_split(df, test_size=test_size, random_state=seed)
        if df_train.shape[0] > max_train_size:
            df_train = df_train.sample(n = max_train_size, random_state=seed)
        if df_test.shape[0] > max_test_size:
            df_test = df_test.sample(n = max_test_size, random_state=seed)
        df_train_list.append(df_train.copy())
        df_test_list.append(df_test.copy())
        del df
    dataset_train = pd.concat(df_train_list)
    dataset_train = dataset_train.sample(dataset_train.shape[0]).reset_index(drop=True).copy()
    del df_train_list
    dataset_test = pd.concat(df_test_list)
    dataset_test = dataset_test.sample(dataset_test.shape[0]).reset_index(drop=True).copy()
    del df_test_list

    y_test = dataset_test[target_col].tolist()
    NT = dataset_train[target_col].value_counts().sort_values().min() 
    # Training step
    cls = RandomForestMC(
        n_trees=64, target_col=target_col, max_discard_trees=256,
        th_start=0.9999, batch_train_pclass = NT // 2, batch_val_pclass = NT // 2
    )
    cls.process_dataset(dataset_train)
    cls.fitParallel(max_workers=16, thread_parallel_method=False)
    modeldict = cls.model2dict()
    dump_file_json(f'/work/data/RandomForestMC_{ds_name}_{seed}.json', modeldict)
    y_pred = cls.testForest(dataset_test)
    F1_M = f1_score(y_test, y_pred, average='macro')
    AUC_ROC_M = roc_auc_score(y_test, y_pred, average='macro')
    RECALL = recall_score(y_test, y_pred, pos_label='1')
    TP_1 = sum([ p == t for p,t in zip(y_pred,y_test) if t == '1'])/y_test.count('1')
    TP_0 = sum([ p == t for p,t in zip(y_pred,y_test) if t == '0'])/y_test.count('0')
    
    Results.append((seed, 'RFMC', F1_M, AUC_ROC_M, RECALL, TP_1, TP_0))
    
    dataset_train_num, dataset_test_num = dfany2dfnumeric(dataset_train[ds_cols], dataset_test[ds_cols])
    # scaler = StandardScaler()
    # scaler.fit(dataset_train_num[ds_cols].to_numpy())  
    # X_train = scaler.transform(dataset_train_num[ds_cols].to_numpy())  
    # X_test = scaler.transform(dataset_test_num[ds_cols].to_numpy())  
    
    cls = RandomForestClassifier(n_estimators=400, n_jobs=-1)
    cls.fit(dataset_train_num[ds_cols].to_numpy(), dataset_train[target_col].to_numpy())
    dump(cls, f'/work/data/RandomForestClassifier_{ds_name}_{seed}.joblib')
    y_pred = cls.predict(dataset_test_num[ds_cols].to_numpy()).tolist()
    F1_M = f1_score(y_test, y_pred, average='macro')
    AUC_ROC_M = roc_auc_score(y_test, y_pred, average='macro')
    RECALL = recall_score(y_test, y_pred, pos_label='1')
    TP_1 = sum([ p == t for p,t in zip(y_pred,y_test) if t == '1'])/y_test.count('1')
    TP_0 = sum([ p == t for p,t in zip(y_pred,y_test) if t == '0'])/y_test.count('0')
    
    Results.append((seed, 'RF', F1_M, AUC_ROC_M, RECALL, TP_1, TP_0))   
    
    cls = MLPClassifier(hidden_layer_sizes=(200,), max_iter=10000)
    cls.fit(dataset_train_num[ds_cols].to_numpy(), dataset_train[target_col].to_numpy())
    dump(cls, f'/work/data/MLPClassifier_{ds_name}_{seed}.joblib')
    y_pred = cls.predict(dataset_test_num[ds_cols].to_numpy()).tolist()
    F1_M = f1_score(y_test, y_pred, average='macro')
    AUC_ROC_M = roc_auc_score(y_test, y_pred, average='macro')
    RECALL = recall_score(y_test, y_pred, pos_label='1')
    TP_1 = sum([ p == t for p,t in zip(y_pred,y_test) if t == '1'])/y_test.count('1')
    TP_0 = sum([ p == t for p,t in zip(y_pred,y_test) if t == '0'])/y_test.count('0')
    
    Results.append((seed, 'MLPC', F1_M, AUC_ROC_M, RECALL, TP_1, TP_0))
    
    del dataset_train, dataset_train_num, dataset_test_num
    
columns=['seed', 'MODEL', 'F1_M', 'AUC_ROC_M', 'RECALL', 'TP_1', 'TP_0']
df_results = pd.DataFrame(Results, columns=columns)
for model in df_results.MODEL.unique():
    md(f'## {model}')
    display(df_results[columns[1:]].query(f'MODEL == "{model}"').describe())
    
df_results.to_csv(f'/work/data/comparison_experiments_{ds_name}.csv', index=False)

Planting the forest:   0%|          | 0/64 [00:00<?, ?it/s]

Planting the forest:   0%|          | 0/64 [00:00<?, ?it/s]

Planting the forest:   0%|          | 0/64 [00:00<?, ?it/s]

Planting the forest:   0%|          | 0/64 [00:00<?, ?it/s]

## RFMC

,F1_M,AUC_ROC_M,RECALL,TP_1,TP_0
count,4.000000,4.000000,4.000000,4.000000,4.000000
mean,0.760039,0.761743,0.723958,0.723958,0.799528
std,0.033205,0.024922,0.074875,0.074875,0.107942
min,0.731386,0.733884,0.666667,0.666667,0.665094
25%,0.732007,0.745381,0.682292,0.682292,0.735849
50%,0.756411,0.761792,0.697917,0.697917,0.820755
75%,0.784443,0.778154,0.739583,0.739583,0.884434
max,0.795947,0.789505,0.833333,0.833333,0.891509


## RF

,F1_M,AUC_ROC_M,RECALL,TP_1,TP_0
count,4.000000,4.000000,4.000000,4.000000,4.000000
mean,0.732046,0.730493,0.661458,0.661458,0.799528
std,0.022256,0.020235,0.011860,0.011860,0.040486
min,0.701923,0.703616,0.645833,0.645833,0.740566
25%,0.721887,0.720568,0.656250,0.656250,0.790094
50%,0.738371,0.735587,0.663194,0.663194,0.813679
75%,0.748530,0.745512,0.668403,0.668403,0.823113
max,0.749518,0.747183,0.673611,0.673611,0.830189


## MLPC

,F1_M,AUC_ROC_M,RECALL,TP_1,TP_0
count,4.000000,4.000000,4.000000,4.000000,4.000000
mean,0.714601,0.712543,0.564236,0.564236,0.860849
std,0.053967,0.049851,0.137654,0.137654,0.059479
min,0.633999,0.637972,0.375000,0.375000,0.773585
25%,0.710588,0.709119,0.520833,0.520833,0.847877
50%,0.738123,0.735161,0.590278,0.590278,0.884434
75%,0.742135,0.738584,0.633681,0.633681,0.897406
max,0.748157,0.741876,0.701389,0.701389,0.900943


# Credit Card Fraude Detection

In [9]:
ds_name = "creditcard_trans_int"
params = dataset_dict[ds_name]
dataset = (
    pd.read_csv(params["csv_path"].replace('~','/work'))[params["ds_cols"] + [params["target_col"]]]
    .dropna()
    .reset_index(drop=True)
)
target_col = params['target_col']
dataset = dataset[params['ds_cols']+[target_col]].dropna().reset_index(drop=True)
dataset[target_col] = dataset[target_col].astype(str)

test_size = 0.5
max_test_size = 10000
max_train_size = 2000

split_seeds = [43, 47, 53, 59]
#def fullCycleAnalysis(dataset,params,test_size_frac, max_test_size, max_train_size, split_seeds, )
target_col = params['target_col']
ds_cols = params["ds_cols"]

Results = []
# Each seed is a experiment.
for seed in split_seeds:
    
    df_train_list = []
    df_test_list = []
    df_classes = [dataset.query(f'{target_col} == "{target_val}"').reset_index(drop=True).copy() for target_val in dataset[target_col].unique()]
    while df_classes:
        df = df_classes.pop(0)
        df_train, df_test = train_test_split(df, test_size=test_size, random_state=seed)
        if df_train.shape[0] > max_train_size:
            df_train = df_train.sample(n = max_train_size, random_state=seed)
        if df_test.shape[0] > max_test_size:
            df_test = df_test.sample(n = max_test_size, random_state=seed)
        df_train_list.append(df_train.copy())
        df_test_list.append(df_test.copy())
        del df
    dataset_train = pd.concat(df_train_list)
    dataset_train = dataset_train.sample(dataset_train.shape[0]).reset_index(drop=True).copy()
    del df_train_list
    dataset_test = pd.concat(df_test_list)
    dataset_test = dataset_test.sample(dataset_test.shape[0]).reset_index(drop=True).copy()
    del df_test_list

    y_test = dataset_test[target_col].astype(str).tolist()
    NT = dataset_train[target_col].value_counts().sort_values().min() 
    # Training step
    cls = RandomForestMC(
        n_trees=64, target_col=target_col, max_discard_trees=256,
        th_start=0.9999, batch_train_pclass = NT // 2, batch_val_pclass = NT // 2
    )
    cls.process_dataset(dataset_train)
    cls.fitParallel(max_workers=16, thread_parallel_method=False)
    modeldict = cls.model2dict()
    dump_file_json(f'/work/data/RandomForestMC_{ds_name}_{seed}.json', modeldict)
    y_pred = cls.testForest(dataset_test)
    F1_M = f1_score(y_test, y_pred, average='macro')
    AUC_ROC_M = roc_auc_score(y_test, y_pred, average='macro')
    RECALL = recall_score(y_test, y_pred, pos_label='1')
    TP_1 = sum([ p == t for p,t in zip(y_pred,y_test) if t == '1'])/y_test.count('1')
    TP_0 = sum([ p == t for p,t in zip(y_pred,y_test) if t == '0'])/y_test.count('0')
    
    Results.append((seed, 'RFMC', F1_M, AUC_ROC_M, RECALL, TP_1, TP_0))
    
    dataset_train_num, dataset_test_num = dataset_train[ds_cols], dataset_test[ds_cols]
    # scaler = StandardScaler()
    # scaler.fit(dataset_train_num[ds_cols].to_numpy())  
    # X_train = scaler.transform(dataset_train_num[ds_cols].to_numpy())  
    # X_test = scaler.transform(dataset_test_num[ds_cols].to_numpy())  
    
    cls = RandomForestClassifier(n_estimators=400, n_jobs=-1)
    cls.fit(dataset_train_num[ds_cols].to_numpy(), dataset_train[target_col].to_numpy())
    dump(cls, f'/work/data/RandomForestClassifier_{ds_name}_{seed}.joblib')
    
    y_pred = cls.predict(dataset_test_num[ds_cols].to_numpy()).tolist()
    F1_M = f1_score(y_test, y_pred, average='macro')
    AUC_ROC_M = roc_auc_score(y_test, y_pred, average='macro')
    RECALL = recall_score(y_test, y_pred, pos_label='1')
    TP_1 = sum([ p == t for p,t in zip(y_pred,y_test) if t == '1'])/y_test.count('1')
    TP_0 = sum([ p == t for p,t in zip(y_pred,y_test) if t == '0'])/y_test.count('0')
    
    Results.append((seed, 'RF', F1_M, AUC_ROC_M, RECALL, TP_1, TP_0))   
    
    cls = MLPClassifier(hidden_layer_sizes=(200,), max_iter=10000)
    cls.fit(dataset_train_num[ds_cols].to_numpy(), dataset_train[target_col].to_numpy())
    dump(cls, f'/work/data/MLPClassifier_{ds_name}_{seed}.joblib')
    y_pred = cls.predict(dataset_test_num[ds_cols].to_numpy()).tolist()
    F1_M = f1_score(y_test, y_pred, average='macro')
    AUC_ROC_M = roc_auc_score(y_test, y_pred, average='macro')
    RECALL = recall_score(y_test, y_pred, pos_label='1')
    TP_1 = sum([ p == t for p,t in zip(y_pred,y_test) if t == '1'])/y_test.count('1')
    TP_0 = sum([ p == t for p,t in zip(y_pred,y_test) if t == '0'])/y_test.count('0')
    
    Results.append((seed, 'MLPC', F1_M, AUC_ROC_M, RECALL, TP_1, TP_0))
    
    del dataset_train, dataset_train_num, dataset_test_num
    
columns=['seed', 'MODEL', 'F1_M', 'AUC_ROC_M', 'RECALL', 'TP_1', 'TP_0']
df_results = pd.DataFrame(Results, columns=columns)
for model in df_results.MODEL.unique():
    md(f'## {model}')
    display(df_results[columns[1:]].query(f'MODEL == "{model}"').describe())
    
df_results.to_csv(f'/work/data/comparison_experiments_{ds_name}.csv', index=False)

Planting the forest:   0%|          | 0/64 [00:00<?, ?it/s]

Planting the forest:   0%|          | 0/64 [00:00<?, ?it/s]

Planting the forest:   0%|          | 0/64 [00:00<?, ?it/s]

Planting the forest:   0%|          | 0/64 [00:00<?, ?it/s]

## RFMC

,F1_M,AUC_ROC_M,RECALL,TP_1,TP_0
count,4.000000,4.000000,4.000000,4.000000,4.000000
mean,0.873081,0.930996,0.872967,0.872967,0.989025
std,0.010804,0.004851,0.009605,0.009605,0.001531
min,0.862976,0.926977,0.865854,0.865854,0.987500
25%,0.864266,0.928027,0.865854,0.865854,0.987950
50%,0.872593,0.929559,0.869919,0.869919,0.988850
75%,0.881409,0.932529,0.877033,0.877033,0.989925
max,0.884164,0.937889,0.886179,0.886179,0.990900


## RF

,F1_M,AUC_ROC_M,RECALL,TP_1,TP_0
count,4.000000,4.000000,4.000000,4.000000,4.000000
mean,0.942143,0.921519,0.844512,0.844512,0.998525
std,0.006279,0.003274,0.006098,0.006098,0.000457
min,0.938067,0.919832,0.841463,0.841463,0.998200
25%,0.938787,0.919869,0.841463,0.841463,0.998275
50%,0.939509,0.919907,0.841463,0.841463,0.998350
75%,0.942865,0.921556,0.844512,0.844512,0.998600
max,0.951488,0.926429,0.853659,0.853659,0.999200


## MLPC

,F1_M,AUC_ROC_M,RECALL,TP_1,TP_0
count,4.000000,4.000000,4.000000,4.000000,4.000000
mean,0.836217,0.926342,0.870935,0.870935,0.981750
std,0.063432,0.009776,0.029006,0.029006,0.011061
min,0.792604,0.913384,0.829268,0.829268,0.973700
25%,0.794005,0.921490,0.862805,0.862805,0.974225
50%,0.812232,0.928991,0.880081,0.880081,0.977900
75%,0.854444,0.933843,0.888211,0.888211,0.985425
max,0.927800,0.934004,0.894309,0.894309,0.997500
